# IMDB Movie Review Classification Using Pre-Trained Embedding

Import tensorflow and other dependencies

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import io

print(tf.__version__)

1.13.1


Download gloVe vectors, these **pre-trained** embedding vectors will be used to initialize the embedding layer

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-04-05 19:57:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-04-05 19:57:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  11.7MB/s    in 87s     

2019-04-05 19:58:44 (9.44 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



Because this is a zip file, we'll have to extract the files first as follows:

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

We'll use the embeddings vectors of dimension 100. We'll read the text file line by line and store the word as key and the embedding vector as it's pair value. This way we'll form our **embedding_index** dictionary.

Example of a line from text file: 

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

In [0]:
embeddings_index = {}
with io.open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embeddings_index[word] = coefs

Download the imdb dataset using **keras.datasets**

In [0]:
imdb = keras.datasets.imdb

Load the imdb dataset using the **load** function. This will return a tuple of numpy arrays in the form **(x_train, y_train), (x_test, y_test)**

S it will return two objects. One will be stored as (train_data, train_labels) and the other one will be stored as (test_data, test_labels)

In [7]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data()

17465344/17464789 [==============================] - 0s 0us/step


Both trianing and test sets have 25000 movie reviews each. The data is evenly split between train and test. 

We'll use 25000 reviews for training and 25000 for testing.

In [8]:
print("Training entries: {}, Training labels:{}".format(len(train_data), len(train_labels)))

Training entries: 25000, Training labels:25000


Each sample/review in the training data is a list of integer values, where each integer represents a word in the vocabulary. 

The label is eitehr 0 or 1, where 0 is a negative review and 1 is a positive review.

In [9]:
print(train_data[0])
print(train_labels[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


The length of different review sis different and so we'll have to **pad** them to make them same length inputs. 

In [10]:
len(train_data[0]), len(train_data[1])

(218, 189)

We need a way to convert integres back to words. Imdb dataset in keras comes with a built in function get_word_index() which has word integer pairs stored. We'll write a function that will take a list of integers (a sample review from training data) and return the text form of that review. 

In [11]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:v+3 for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1646592/1641221 [==============================] - 0s 0us/step


In [12]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [15]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [16]:
print(train_data[0])

[    1    14    22    16    43   530   973  1622  1385    65   458  4468
    66  3941     4   173    36   256     5    25   100    43   838   112
    50   670 22665     9    35   480   284     5   150     4   172   112
   167 21631   336   385    39     4   172  4536  1111    17   546    38
    13   447     4   192    50    16     6   147  2025    19    14    22
     4  1920  4613   469     4    22    71    87    12    16    43   530
    38    76    15    13  1247     4    22    17   515    17    12    16
   626    18 19193     5    62   386    12     8   316     8   106     5
     4  2223  5244    16   480    66  3785    33     4   130    12    16
    38   619     5    25   124    51    36   135    48    25  1415    33
     6    22    12   215    28    77    52     5    14   407    16    82
 10311     8     4   107   117  5952    15   256     4 31050     7  3766
     5   723    36    71    43   530   476    26   400   317    46     7
     4 12118  1029    13   104    88     4   381   

Create a weight matrix for the words in the training doc

In [17]:
vocab_size = 88588

print(len(word_index))

embeddings_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embeddings_matrix[i] = embedding_vector

88588


Define the embedding layer:

input as 88588(number of words in vocabulary)

output shape as 100 because each embedding will be of dim 100

Weights as our pretrained embedding matrix

trainable as false, because we are not training this layer

In [0]:
e = keras.layers.Embedding(vocab_size, 100, weights=[embeddings_matrix], trainable=False)

**Define the model structure:**

Add an average pooling layer

Add a fully connected layer with 100 hidden units

Add a fully connected layer with 16 hidden units

Add a single hidden unit, this is the output node, we'll use sigmoid activation


**Output:**

1 - positive review

0 - negative review


In [21]:
# input shape is the vocabulary count used for the movie reviews (88588 words)

model = keras.Sequential()
model.add(e)
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(100, activation=tf.nn.relu))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         8858800   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1616      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 8,870,533
Trainable params: 11,733
Non-trainable params: 8,858,800
_________________________________________________________________


**Compile the model:**

Define loss as binary_crossentropy

Define optimizer as adam

Define metrics as accuracy

In [0]:
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics = ["acc"])

**Split train data:**

Split train data into train and validation sets

In [0]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Train the model for **500** epochs with **512** batch size and store the history

In [24]:
history = model.fit(partial_x_train, partial_y_train, epochs=500, batch_size=512, validation_data=(x_val,y_val), verbose=1)

Train on 15000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
15000/15000 [==============================] - 1s 98us/sample - loss: 0.6805 - acc: 0.6262 - val_loss: 0.6642 - val_acc: 0.6634
Epoch 2/500
15000/15000 [==============================] - 1s 65us/sample - loss: 0.6496 - acc: 0.6618 - val_loss: 0.6335 - val_acc: 0.6819
Epoch 3/500
15000/15000 [==============================] - 1s 63us/sample - loss: 0.6147 - acc: 0.6905 - val_loss: 0.6037 - val_acc: 0.6955
Epoch 4/500
15000/15000 [==============================] - 1s 62us/sample - loss: 0.5859 - acc: 0.7076 - val_loss: 0.5756 - val_acc: 0.7144
Epoch 5/500
15000/15000 [==============================] - 1s 62us/sample - loss: 0.5577 - acc: 0.7321 - val_loss: 0.5464 - val_acc: 0.7413
Epoch 6/500
15000/15000 [==============================] - 1s 61us/sample - loss: 0.5376 - acc: 0.7461 - val_loss: 0.5322 - val_acc: 0.7544
Epoch 7/500
15000/15000 [==============================] - 1s 

**Evaluate the model on test data:**

In [25]:
results = model.evaluate(test_data, test_labels)

25000/25000 [==============================] - 1s 51us/sample - loss: 0.4889 - acc: 0.7828


In [26]:
print(results)

[0.4888888516235352, 0.7828]


In [28]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])